In [43]:
function filter_jl_files(dir_list::Vector{String})
    jl_files = filter(x -> endswith(x, ".jl"), dir_list)
    non_jl_files = setdiff(dir_list, jl_files)
    return jl_files, non_jl_files
end

function add_filepaths!(filepaths::Dict{String, String}, dir_path::String, dir_list::Vector{String})
    for file in dir_list
        filepaths[file] = joinpath(dir_path, file)
    end
end

function find_jl_files(dir_path::String)
    filepaths = Dict{String, String}()
    jl_files, non_jl_files = filter_jl_files(readdir(dir_path))
    add_filepaths!(filepaths, dir_path, jl_files)
    new_dirs = filter(x -> isdir(joinpath(dir_path, x)), non_jl_files)
    for new_dir in new_dirs
        merge!(filepaths, find_jl_files(joinpath(dir_path, new_dir)))
    end
    return filepaths
end

find_jl_files (generic function with 1 method)

In [44]:
function clean_file(dir_path::String)
    open(dir_path, "r") do file
        output_lines = ""
        long_comment = false
        for line in eachline(file)
            # Check for doc strings and ignore them
            if contains(line, """\"\"\"""")
                long_comment = !long_comment
            end
            if long_comment
                continue
            else 
                # Remove comments
                if !startswith(strip(line), "#") && line != ""
                    # Remove tabs
                    line = replace(line, "\t" => " ")
                    # Make all one line, but add space to avoid concatenating words
                    output_lines = string(output_lines, " ", line)
                end
            end
        end
        return output_lines
    end
end

function make_filestrings(filepath::String)
    filestrings = Dict{String, String}()
    for (filename, filepath) in filepaths
        filestrings[filename] = clean_file(filepath)
    end
    return filestrings
end

make_filestrings (generic function with 1 method)

In [45]:
function filter_exp_var!(exp_and_vars::Dict{String, Dict{String, String}}, line::String)
    if contains(line, "@variable")
        var_name = get_var_name(line)
        exp_and_vars["var"][var_name] = filename
    elseif contains(line, "@expression")
        exp_name = get_exp_name(line)
        exp_and_vars["exp"][exp_name] = filename
    end
end

function extract_model_name(filestring::AbstractString, filter_prefix::AbstractString)
    x = split(filestring, ",")
    # Try to extract the name based on a prefix
    x2 = split(x[2], " ")
    x2 = filter(x -> startswith(x, filter_prefix), x2)
    if !isempty(x2)
        x = x2[1]
    else
        x = x[2]
    end
    # Remove any indexing
    if contains(x, "[")
        x = split(x, "[")[1]
    end
    # Remove whitespace
    return strip(x)
end

function filter_filestring(filestring::String, filter_string::String)
    cases = split(filestring, filter_string)
    outputs = Vector{String}()
    filter_prefix = string(filter_string[2])
    for i in 2:length(cases)
        push!(outputs, extract_model_name(cases[i], filter_prefix))
    end
    return outputs
end

filter_filestring (generic function with 1 method)

In [46]:
function find_exp_var_created(filepaths::Dict{String, String})
    # exp_and_vars = Dict{String, Dict{String, String}}("var" => Dict{String, String}(), "exp" => Dict{String, String}())
    exp_and_vars = Dict("var" => Dict{String, Any}(), "exp" => Dict{String, Any}())
    for (filename, filepath) in filepaths
        # println(filename)
        filestring = clean_file(filepath)
        new_var = filter_filestring(filestring, "@variable")
        for var in new_var
            if !haskey(exp_and_vars["var"], var)
                exp_and_vars["var"][var] = Dict("created" => filename, "accessed" => Vector{String}())
            end
            exp_and_vars["var"][var]["created"] = filename
        end
        new_exp = filter_filestring(filestring, "@expression")
        for exp in new_exp
            if !haskey(exp_and_vars["exp"], exp)
                exp_and_vars["exp"][exp] = Dict("created" => filename, "accessed" => Vector{String}())
            end
            exp_and_vars["exp"][exp]["created"] = filename
        end
    end
    return exp_and_vars
end

find_exp_var_created (generic function with 1 method)

In [47]:
function check_access(filepaths::Dict{String, String}, target_names::Vector{String})
    file_accesses = Dict{String, Vector{String}}()
    for (filename, filepath) in filepaths
        filestring = clean_file(filepath)
        file_accesses[filename] = Vector{String}()
        for target_name in target_names
            if contains(filestring, target_name)
                push!(file_accesses[filename], target_name)
            end
        end
    end
    return file_accesses
end

check_access (generic function with 1 method)

In [48]:
function push_accesses!(exp_or_vars::Dict{String, Any}, accesses::Dict{String, Vector{String}})
    for (filename, model_names) in accesses
        for model_name in model_names
            push!(exp_or_vars[model_name]["accessed"], filename)
        end
    end
end

push_accesses! (generic function with 1 method)

In [49]:
function get_all_accesses!(exp_and_vars::Dict{String, Dict{String, Any}}, filepaths::Dict{String, String})
    var_accessed = check_access(filepaths, collect(keys(exp_and_vars["var"])))
    push_accesses!(exp_and_vars["var"], var_accessed)
    exp_accessed = check_access(filepaths, collect(keys(exp_and_vars["exp"])))
    push_accesses!(exp_and_vars["exp"], exp_accessed)
end

get_all_accesses! (generic function with 1 method)

In [50]:
function make_table_header(headers::Set{String}, model_type::String)
    fun_headers = copy(headers)
    table_header = "|$(model_type) name"
    table_divider = "|:-"
    if "created" in fun_headers
        table_header = string(table_header, "|created")
        table_divider = string(table_divider, "|:-")
        delete!(fun_headers, "created")
    end    
    for header in fun_headers
        table_header = string(table_header, "|", header)
        table_divider = string(table_divider, "|", ":-")
    end
    table_header = string(table_header, "|")
    table_divider = string(table_divider, "|")
    return table_header, table_divider
end

make_table_header (generic function with 1 method)

In [51]:
function make_table_row(model_name::String, model_dict::Dict{String, Any}, headers::Set{String})
    table_row = "|$(model_name)"
    fun_headers = copy(headers)
    if "created" in fun_headers
        table_row = string(table_row, "|", model_dict["created"])
        delete!(fun_headers, "created")
    end
    for header in fun_headers
        table_row = string(table_row, "|", join(model_dict[header], ", "))
    end
    table_row = string(table_row, "|")
    return table_row
end

make_table_row (generic function with 1 method)

In [52]:
function data_2_markdown(exp_or_vars::Dict{String, Any}, model_type::String, headers::Set{String})
    table = Vector{String}()
    push!(table, "## $(model_type)s")
    table_header, table_divider = make_table_header(headers, model_type)
    push!(table, table_header)
    push!(table, table_divider)
    sorted_names = sort(collect(keys(exp_or_vars)))
    # for (name, data) in exp_or_vars
    for name in sorted_names
        push!(table, make_table_row(name, exp_or_vars[name], headers))
    end
    return table
end

data_2_markdown (generic function with 1 method)

In [53]:
# Get all Julia files in the src directory
src_path = joinpath(dirname(pwd()), "src")
filepaths = find_jl_files(src_path)

# Find where all the Variables and Expressions are defined
exp_and_vars = find_exp_var_created(filepaths)

# Find where each Variable is accessed
get_all_accesses!(exp_and_vars, filepaths)
headers = Set{String}(["created", "accessed"])

var_table = data_2_markdown(exp_and_vars["var"], "Variable", headers)
exp_table = data_2_markdown(exp_and_vars["exp"], "Expression", headers)

open(joinpath(pwd(), "model_table.md"), "w") do io
    println(io, "# Model Variables and Expressions")
    for line in var_table
        println(io, line)
    end
    for line in exp_table
        println(io, line)
    end
end

# Find where the Objective is defined, and each of the components of the Objective

In [54]:
filename = "thermal_commit.jl"
filepath = filepaths[filename]
filestring = clean_file(filepath)
q = split(filestring, "function ")
q = q[2:end]
for w in q
    # w = q[1]
    w = split(w, "(")
    fun_name = w[1]
    fun_args = split(split(w[2], ")")[1], ", ")
    println(fun_name)
    for a in fun_args
        println(a)
    end
end

thermal_commit
EP::Model
inputs::Dict
Reserves::Int
thermal_commit_reserves
EP::Model
inputs::Dict
hoursbefore
p::Int
t::Int
b::UnitRange{Int}


In [57]:
function find_functions(filepaths::Dict{String, String})
    function_names = Dict{String, String}()
    for (filename, filepath) in filepaths
        filestring = clean_file(filepath)
        new_functions = filter_filestring(filestring, "function ")
        for fun in new_functions
            if !haskey(function_names, fun)
                function_names[fun] = filename
            end
        end
    end
    return function_names
end

find_functions (generic function with 1 method)